In [1]:
from src.requirements import *
from src.ssl_model import *
from src.asr_model import *
from src.inference_model import *
from src.tokenizer import Tokenizer
import jiwer

In [2]:
ssl_ver = 100_000
asr_ver = 80_000

device = "cuda" if torch.cuda.is_available() else "cpu"

ssl_path = os.path.join("models", "ssl_model", f"ssl_model_prototype_{ssl_ver}.pth")
asr_path = os.path.join("models", "asr_model", f"asr_model_prototype_{asr_ver}.pth")
tokenizer_path = os.path.join("data", "tokenizer.json")

metadata = pd.read_csv(os.path.join('data', 'metadata.tsv'), sep='\t')

In [3]:
# tokenizer = Tokenizer.load(tokenizer_path)

In [4]:
infer_model = InferenceModel(ssl_path, asr_path, tokenizer_path, Tokenizer, device)

Final Vocabulary Size after filtering: 494
Blank ID: 0


In [5]:
num = torch.randint(low=0, high=metadata.shape[0], size=(10,1))
references = []
hypotheses = []

In [6]:
for idx in num:
    reference = metadata['transcript'].tolist()[idx]
    path = metadata['path'].tolist()[idx]
    waveform, sr = sf.read(path, always_2d=True)

    text = infer_model(waveform, sr)
    references.append(reference)
    hypotheses.append(text)

In [7]:
def calc_cer(references, hypotheses):
    cer = jiwer.cer(references, hypotheses)
    return cer

print(f"Character Error Rate: {calc_cer(references, hypotheses)*100:0.4f}%")

Character Error Rate: 99.4152%


In [8]:
references

['मृत्यु हुन सक्छ',
 'पाल्पाको जुठा पौवासित',
 'अपभ्रंश भएर पौराही',
 'अदालतले दिल्लीको सबै',
 'गर्ने र कहिल्यै',
 'हिमकला मन्दिर हो',
 'प्रस्तुत गरेका थिए',
 'वसुन्धरा भुसाल देखिएपछि',
 'दूत वाक्यका',
 'उम्रने गर्दछन्']

In [9]:
hypotheses

['', '  ', 'स', '   ', '', '', 'स', ' ', ' ', '']

In [ ]:
for hypo in hypotheses:
    pred = torch.argmax(hypo, dim=-1)
    print(pred)

In [ ]:
result = []
prev = None
blank_id = 0

In [ ]:
pred